# Model compression survey

## Introduction 
DNN models are computationlly expensive and memory intensive, 
the model compression way is used to accelerate the deep network without significantly decreasing the model performance.

The model compression approaches can be classified into four categories:
    
- parameter pruning and sharing
    - reducing redundant parameters which are not sensitive to the performance
- low-rank factorization
    - using matrix/tensor decomposition to estimate the informative parameters
- transferred/compact convolutional filters
    - designing special structural convolutional filters to save parameters
- knowledge distillation
    - training a compact neural network with distilled knowledge of a large model

We will dive into the details of these four methods

## Parameter pruning and sharing

This technology can be further classified into three categories:

- model quntization and binarization
- parameter sharing 
- structural matrix

### Quantization and Binarization

*reference: Improving the speed of neural networks on CPUs*

#### Floating-point implementation (competitive with Eigen in small matrice mulplication)

##### Loop unrolling and parallel accumulators gets 26% speed improvement

A matrix multipication is as follows

```c++
// two matrice
// matrix A's shape is (m, n)
float **A;
// matrix B's shape is (n, t) 
float **B;
// A multiply B will get a new matrix of shape (m, t)
float **C;
for (int i = 0; i < m; i++) {
    for (int j = 0; j < n; j++) {
        C[i][j] = 0.;
        for (int k = 0; k < t; k++) { // NOTE this for-loop takes much time to check termination
            c[i][j] += A[i][k] * B[j][k];
        }
    }
}
```

**To reduce the overhead of checking for-loop termination, one can unroll the computation by accumulating elements at a time.**

```c++
auto& c = C[i][j];
auto* a = A[i];
auto* b = B[j];
C += a[k]*b[k] + a[k+1]*b[k+1] + a[k+2]*b[k+2] + a[k+3]*b[k+3];
```

a second technique is to **use multiple accumulators in parallel, 
the compiler might optimize it in pipelining operations.** for example

```c++
c0 += a[i] * b[i];
c1 += a[i+1] * b[i+1];
c2 += a[i+2] * b[i+2];
c3 += a[i+3] * b[i+3];
c = c0 + c1 + c2 + c3;
```

##### SIMD to make low-level parallelization

On Intel and AMD CPUs of the x86 family, the SIMD instructions typically operate on 16 bytes worth of data at a time, that is 2 doubles, 4 floats, 8 shorts or 16 bytes at a time.

<p align="center">
    <img src="./images/simd.png/" style="width: 600px;">
</p>

The code 

```c++
c += a[k]*b[k] + a[k+1]*b[k+1] + a[k+2]*b[k+2] + a[k+3]*b[k+3];
```
which is a sum of 4 floats can be rewritten to

```c++
#include <mmintrin.h>
__m128 c = _mm_add_ps(a, b);
```

##### SIMD and SSE2  to perform multiply-and-add

Here is a simplified example accumulating the scalar product of `__m128 *a` and `__m128 *b` to `__m128 sum`

```c++
// c[0] = a[0]*b[0], ..., c[3] = a[3]*b[3]
__m128 c = _mm_mul_ps(*a, *b);
// sum[0] += c[0], ..., sum[3] += c[3]
sum = _mm_add_ps(c, sum);
```

#### Fixed-point implementation